In [100]:
import xgboost
import joblib
import numpy as np
import pandas as pd

#### Load model, feature names, testing data and counterfactuals from testing set

In [101]:
#model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted')
model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted_clip')
#model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted_taper')
#model = joblib.load('files/model_asap7_776_counter_both_ori_adjusted_late_taper')

feature_names = joblib.load('files/feature_names_776')

data_test_ori = pd.read_csv('ASAP7 Test Set.tsv', sep='\t')

feat_test_ori = joblib.load('files/asap7_features_test_776')

feats_counter_test_776 = joblib.load('files/feats_counter_test_correct_776')
np.array(feats_counter_test_776).shape

(100, 31, 1, 776)

#### Get the scores of the original essays

In [102]:
scores_ori = []
row_ctr = 0

for feat in feat_test_ori[:100]:
    dmat = xgboost.DMatrix(feat.reshape(1,-1), feature_names=feature_names)
    score = model.predict(dmat)
    scores_ori.append(score[0])
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [103]:
joblib.dump(scores_ori, "scores_ori_test_776_both_adjusted")
#scores_ori = joblib.load("scores_ori_test")

['scores_ori_test_776_both_adjusted']

#### Predict the scores of the counterfactuals

In [104]:
scores_counter = []

row_ctr = 0

for row in feats_counter_test_776:
    row_score = []
    for essay in row:
        essay = xgboost.DMatrix(essay, feature_names=feature_names)
        score = model.predict(essay)
        row_score.append(score[0])
    scores_counter.append(row_score)
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [105]:
#joblib.dump(scores_counter, "scores_counterfactual_test_newmodel_both")
#scores_counter = joblib.load('scores_counterfactual_test_newmodel_both')

In [106]:
np.array(scores_counter).shape

(100, 31)

#### Compare the scores Original vs Counterfactual

In [107]:
comparison = []

ctr = 1

for idx, row_scores in enumerate(scores_counter):
    ori_score = scores_ori[idx]
    compare_row = []
    for score in row_scores:

        if ctr <= 10:
            print(ctr)
            print("ori: ", ori_score)
            print("counter: ", score)
        
        score_diff = abs(score - ori_score)
        if score >= ori_score:
            compare_row.append((True, score_diff, score))
        else:
            compare_row.append((False, score_diff, score))
        ctr = ctr + 1
        
    comparison.append(compare_row)

1
ori:  13.925055
counter:  21.956097
2
ori:  13.925055
counter:  22.039719
3
ori:  13.925055
counter:  22.86224
4
ori:  13.925055
counter:  23.328743
5
ori:  13.925055
counter:  20.081861
6
ori:  13.925055
counter:  22.1449
7
ori:  13.925055
counter:  23.04682
8
ori:  13.925055
counter:  23.636206
9
ori:  13.925055
counter:  24.187834
10
ori:  13.925055
counter:  22.836412


In [108]:
comparison

[[(True, 8.031042, 21.956097),
  (True, 8.114664, 22.039719),
  (True, 8.937185, 22.86224),
  (True, 9.403688, 23.328743),
  (True, 6.156807, 20.081861),
  (True, 8.219845, 22.1449),
  (True, 9.121765, 23.04682),
  (True, 9.711151, 23.636206),
  (True, 10.262779, 24.187834),
  (True, 8.911358, 22.836412),
  (True, 8.470005, 22.39506),
  (True, 5.6012115, 19.526266),
  (True, 9.561327, 23.486382),
  (True, 6.156807, 20.081861),
  (True, 6.7305984, 20.655653),
  (True, 9.743683, 23.668737),
  (True, 9.112726, 23.03778),
  (True, 9.307665, 23.23272),
  (True, 10.113409, 24.038464),
  (True, 10.2847805, 24.209835),
  (True, 10.450657, 24.375711),
  (True, 9.287674, 23.212729),
  (True, 5.6377106, 19.562765),
  (True, 6.796322, 20.721376),
  (True, 6.7791843, 20.704239),
  (True, 9.89069, 23.815744),
  (True, 9.642998, 23.568052),
  (True, 9.899759, 23.824814),
  (True, 10.564068, 24.489122),
  (True, 6.796322, 20.721376),
  (True, 10.446285, 24.37134)],
 [(True, 4.690336, 20.727316),
  (Tr

#### Count the number of False and mean of score difference for the 'False' predictions

In [109]:
# Initialize counters and lists to hold False values' second column values
false_count = 0
false_values = []
new_label = []
true_count = 0

# Iterate through each sublist
for idx, sublist in enumerate(comparison):
    ori = scores_ori[idx]
    row_label = []
    for value in sublist:

        formula = ori + value[1]
        
        if value[0] == False:  # If the first element is False
            false_count += 1
            false_values.append(value[1])  # Collect the second element (the difference)
            row_label.append(formula)
        else:
            true_count += 1
            row_label.append(value[2])
    new_label.append(row_label)
    
# Calculate the mean of the differences for False values only
if false_count > 0:
    mean_difference = np.mean(false_values)
else:
    mean_difference = 0  # Handle case where there are no False values

print(f"Count of False values: {false_count}")
print(f"Count of True values: {true_count}")
print(f"Mean of the differences for False values: {mean_difference}")


Count of False values: 504
Count of True values: 2596
Mean of the differences for False values: 2.952366352081299


In [110]:
np.array(comparison).shape

(100, 31, 3)

In [111]:
acc = (true_count / 3100) * 100
acc

83.74193548387096

In [87]:
np.array(new_label).shape

(100, 31)

In [14]:
#joblib.dump(new_label, 'new_label_test')